# Data Science at UCSB

# Python for Data Science: Tabular Data

## Jason Freeberg, Fall 2016 

Tabular data is how a lot data is organized. It is not the *only* data format, but it is the easiest to work with because it is well-structured. Other data formats you will come across include [JSON](http://www.json.org/), [Relational and Non-Relational Databases](https://www.mongodb.com/scale/relational-vs-non-relational-database), images, and audio files. And believe it or not you are already familiar with tabular data, it's simply a table with columns and rows. Just like in Excel.

As data scientists to-be, however, we need to make define some terms. We will often refer to rows as *observations* or *records*, and columns as *variables* or *features*. The *header* is the top row containing the names of our variables. In the example below our variables are country, salesperson, order id, and so on. Our observations are individual orders with those variable values. Our header, in this case, would be the row with index #1.

![data_pic](http://mothimages.s3.amazonaws.com/tabular_data_1.png)

In today's lab we will get acquainted with the [pandas module](http://pandas.pydata.org/) by loading a Comma Seperated Value (.csv) [file](https://archive.ics.uci.edu/ml/datasets/Forest+Fires) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.html). We will then check it, coerce the variables to the correct format, check for missing values, and create aggregate reports by conditional selection. Then you'll follow the same pipeline on your own with a different dataset!


In [58]:
# Import the modules we'll need and assign the data's URL.

# By the way, it's customary to include all module imports at the beginning of your script.
import numpy as np
import pandas as pd
from urllib.request import urlopen

UCI_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv'

In [59]:
# A helper function to read the data from the url. 
# Just run this cell, but understand what the function is doing.


def read_csv_from_url(URL):
    """
    Takes as input a string containing the URL pointing to a dataset from the UCI data repository.
    Returns a pandas dataframe containing the data with some columns coerced as strings.
    """
    
    response = urlopen(URL)
    lines = pd.read_csv(response, 
                        header = 0,
                        index_col = False,
                        dtype = {'DMC' : str,
                                 'temp' : str,
                                 'area' : str})
    
    return pd.DataFrame(lines)

# Load and Check Data

Using the URL and function above, let's load the data into our notebook as a pandas dataframe. We will then inspect dataframe's size, missing values, and variable types.

In [60]:
# Load the data and print the head of the dataframe.
fire_df = read_csv_from_url(UCI_data_URL)

# Let's check the head and size of our data
print("Number of rows:", fire_df.shape[0])
print("Number of columns:", fire_df.shape[1])
print(fire_df.head())

Number of rows: 517
Number of columns: 13
   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0    0
1  7  4   oct  tue  90.6  35.4  669.1  6.7    18  33   0.9   0.0    0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0    0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2    0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0    0


We now have our data loaded and assigned as a pandas.DataFrame object. However, I made a *slight* adjustment and loaded some variables as **strings**. Know that the pandas DataFrame( ) method is very well built and could have inferred the correct types for all columns, but variable coersion is a common data preparation task so we will do it in this lab.

**Right now we'll check for missing and incorrect values.**

In [61]:
# .isnull() returns a dataframe of logical (T/F) entries where True = Is_Null and False = Not_Null.
# We can use the sum() method to take the sums by each column. Remember that True = 1, False = 0.

logical_dataframe = fire_df.isnull()
print(logical_dataframe.sum())

X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
area     0
dtype: int64


So we don't have any **NaN** or **None** values in our columns. But we're not out of the woods yet. Let's take a look at our categorical variables and check that they're reasonable. By printing out the unique strings in each column, we'll be able to see if there are any inappropriate values like misspelled days or months.

In [62]:
# The syntax, "dataFrame.columnName" will return a pandas Series object. 
# We can use the unique() method to get the distinct strings held in the Series object. 

print("Class of our returned column:", type(fire_df.month))
print(fire_df.month.unique())
print(fire_df.day.unique())

Class of our returned column: <class 'pandas.core.series.Series'>
['mar' 'oct' 'aug' 'sep' 'apr' 'jun' 'jul' 'feb' 'jan' 'dec' 'may' 'nov']
['fri' 'tue' 'sat' 'sun' 'mon' 'wed' 'thu']


Luckily for us the UCI datasets are often very clean. Although we didn't uncover any missing or incorrect values in this dataset, these types of checks will become routine when you start a project or intern at a company.

**Now we'll look at our column types and make adjustments as necessary.**

In [63]:
# This will show our columns and their corresponding types.
print('The data types of all features:')
print(fire_df.dtypes, '\n')

# That's a lot to look at, let's narrow our search. This is a conditional selection, which we'll get to later.
print('Our non-numeric variables:')
print(fire_df.dtypes[fire_df.dtypes == 'object'], '\n') # Condition is in the square brackets.

# Month and day are okay being objects (strings), but those other three need to be converted to floats...
fire_df.DMC = fire_df.DMC.astype(float)
fire_df.area = fire_df.area.astype(float)
fire_df.temp = fire_df.temp.astype(float)

print('---------- After Type Coercion ----------', '\n')

print(fire_df.dtypes[fire_df.dtypes == 'object'])

The data types of all features:
X          int64
Y          int64
month     object
day       object
FFMC     float64
DMC       object
DC       float64
ISI      float64
temp      object
RH         int64
wind     float64
rain     float64
area      object
dtype: object 

Our non-numeric variables:
month    object
day      object
DMC      object
temp     object
area     object
dtype: object 

---------- After Type Coercion ---------- 

month    object
day      object
dtype: object


# Conditional Selection

Now that we have vetted the data for discrepancies, we can create do some exploratory analysis. Let's first cover conditional selection. Our data is 517 x 13, but we often won't want to use the entire table all the time. We might only need a couple columns, or perhaps we only want to look at the data on Tuesdays. With pandas, it's easy to select columns and rows based on arbitrary conditions. 

- Here's the basic syntax: *dataframe*[*condition on **rows***]\[*names or numbers of **columns***]
- Alternatively... *dataframe*.ix[*condition on **rows***, *selection of **columns*** ]
  - If you come from an R background, the .ix attribute syntax may seem familiar
- use *dataframe*.iloc**[ ]** for selecting rows and columns based on **purely numerical** indices

Click [here](http://pandas.pydata.org/pandas-docs/stable/indexing.html) for the full documentation on slicing and dicing pandas DataFrames.

In [64]:
# Filter rows...
fire_df[:5]  # prints first ten rows (from 0 to 9)
fire_df.ix[:5, :] # same thing
fire_df.iloc[:5]  # same as above
fire_df.iloc[[1,2,3,4,5]]  # same again
fire_df[fire_df.area > 30]
fire_df[ (fire_df.area > 30) & (fire_df.rain > 10) ]  # two conditions on rows

# Select columns...
fire_df.temp  # a single column
fire_df["temp"]  # also a single column 
fire_df.ix[:, ['temp']] # same as above
fire_df[['day', 'area', 'rain']]  # multiple columns

# Select columns AND filter rows...
fire_df[fire_df.area > 30][['day', 'area', 'rain']]

# Using '.ix' and making the same selection as above...
fire_df.ix[ fire_df.area > 30, ['day', 'area', 'rain'] ]

# Just to hide all output...
print("Golly Gee! Python is so cool!")

Golly Gee! Python is so cool!


# Your turn

Get in the driver's seat, because it's your turn to write some code. Look for the &lt;FILL IN&gt; bits. 

Good luck and be sure to ask Jason for clarification or help.

In [65]:
# URL of a different dataset
newURL = "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"

# Read the data from the URL, turn it into a pandas dataframe, and return it
def myCSVReader(a_URL):
    response = <FILL IN>(<FILL IN>)
    lines = pd.<FILL IN>(<FILL IN>)
  
    return <FILL IN>

carData = myCSVReader(newURL)

carData.head()

,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [66]:
# Uh oh! Looks like our data is missing something, but what?
# ... We're missing the header of the data (the column names)!
# Here they are.

colNames = ["symboling", "normalized_losses", "make", "fuel_type", 
            "aspiration", "num_doors", "body_style", "drive_wheels", 
            "engine_location", "wheelbase", "length", "width", "height", 
            "curb_weight", "engine_type", "num_cylinders", "engine_size", 
            "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", 
            "peak_rpm", "city_mpg", "highway_mpg", "price"]

# Now edit our CSV reader to accept a list of column names and then use it to set the DataFrame's column names.
# Look around online to find the DataFrame attribute you need to set ;)

def myCSVReader(a_URL, column_names):
    response = urllib2.urlopen(a_URL)
    data = pd.read_csv(response)
    data.columns = <FILL IN>
  
    return pd.DataFrame(<FILL IN>)

carData = myCSVReader(newURL, colNames)

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheelbase,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


In [67]:
# Now let's practice some conditional selection

# Select rows 1, 2, 3 using numeric indices -- all columns
test1 = carData.iloc[<FILL IN>]

# Select columns 4, 5, 6 using column numbers -- all rows
test2 = carData.iloc[<FILL IN>, <FILL IN>

# Subset the DataFrame to only sedans
test3 = carData[<FILL IN> == <FILL IN>]

# Now in those convertible cars, only keep the horsepower, engine size, city MPG, and highway MPG.
test4 = test3[[<FILL IN>]]

# Now find the mean horsepower!
test4['horsepower'].<FILL IN>()

1.0636990636472928e+234

In [68]:
# Uh-oh! That doesn't seem like a reasonable average horsepower for sedans!
# There must be something fishy in the data; let's investigate.

# Use .isnull() and .sum() to check for missing values by colum
print( carData.<FILL IN> )

# Hmmm... interesting. Print all unique values in the horsepower column
print( carData[<FILL IN>].<FILL IN>() )

array(['111', '154', '102', '115', '110', '140', '160', '101', '121',
       '182', '48', '70', '68', '88', '145', '58', '76', '60', '86', '100',
       '78', '90', '176', '262', '135', '84', '64', '120', '72', '123',
       '155', '184', '175', '116', '69', '55', '97', '152', '200', '95',
       '142', '143', '207', '288', '?', '73', '82', '94', '62', '56',
       '112', '92', '161', '156', '52', '85', '114', '162', '134', '106'], dtype=object)

In [69]:
# Ah-Ha! See the problem?! Let's see if that issue is in other columns as well.
# Do the same check for normalized_losses
print( carData[<FILL IN>].<FILL IN> )

# Yup! The '?' must be their indicator for a missing value. 
# Let's replace '?' with np.nan in all rows
carData.replace(to_replace = <FILL IN>, value = <FILL IN>, inplace = True)

# Now let's check the number of missing values again
print( carData.<FILL IN>.<FILL IN> )

['?' '164' '158' '192' '188' '121' '98' '81' '118' '148' '110' '145' '137'
 '101' '78' '106' '85' '107' '104' '113' '150' '129' '115' '93' '142' '161'
 '153' '125' '128' '122' '103' '168' '108' '194' '231' '119' '154' '74'
 '186' '83' '102' '89' '87' '77' '91' '134' '65' '197' '90' '94' '256' '95']


symboling             0
normalized_losses    40
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheelbase             0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 4
dtype: int64

In [73]:
# Now that the '?' characters have been replaced, we need to change the data types for some columns.

# Make a list of the horsepower, normalized_losses, prince, and peak_rpm column names
stringToNumeric = [<FILL IN>]

# Iterate over those columns and change the type
for column in stringToNumeric:
    carData[<FILL IN>] = carData[<FILL IN>].astype(np.float64)

# Print the datatypes again to check
print(carData.<FILL IN>)

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheelbase            float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                  object
stroke                object
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object


In [75]:
# Now let's finally get that mean horsepower for sedans!

carData[<FILL IN>].<FILL IN>

104.22277227722772

Great! Use any remaining time to work on DataCamp or Codecademy exercises.